# Import data

Importing and parsing the XML-file 

In [112]:
# Importing packages
import xml.etree.ElementTree

#Parsing XML file and getting root
XML_parsed = xml.etree.ElementTree.parse('00019.xml')
root=XML_parsed.getroot()

In [113]:
#Example of how to get the values from a tag. Below we get the values from the <mnemonicList> tag
root.find(".//{http://www.witsml.org/schemas/1series}mnemonicList").text

'TIME,SRVTVD,GS_SPM1,GS_TV11,SRV_HY,HKLD30s,UTSTAT,SWOB30s,GS_TV03,SPPA,ETSL,AJAM_MWD,ATMP_RT,PMIN_TM_RT,HKLD,GS_G_C1,GS_TVCA,DNI_MP,BPOS,GS_DVER,HKLN,GS_TV06,GS_TV05,SGX,GS_SWOB,PMAX_TM_RT,GS_DRTV,CRS_CNAZI,DBTV,CUREDT_RT,GS_G_C2,STK2,SHK2_RT,APRS_RT,TTONB,GS_TV01,SHX,GS_ACTC,SGZ,PMIN_RT,SRVEW,INCL_CONT_RT,SPM1,GS_TV13,BHFG,BVEL,TFLO,TOTAL_CORR,GS_G_NC5,MSP4,PESD_TM_RT,TOJ,ACTC,CPOF,RSUX,HKLX,STK1,EDRT,GS_DCHM,GS_TV04,ECD_MW_IN,TEMP_DNI_RT,GS_TV08,NRPM_RT,THKD,PASS_NAME,TOFF,CRS4,GS_DMEA,SRV_HX,SRVAZI,GS_G_NC4,TEFNEG,A40H_UNC_RT,ANGLX,SGY,SDEP_CONT_RT,GS_CFIA,GS_DBTM,GS_DRTM,TCHR,HKLI,ESD_RT,SRV_GX,GS_CHKP,SHY,GS_HKLD,RHX,MBOT,FVOC,SHKTOT_RT,GS_MTOA,ESD_MAX_RT,ROP,A40H_COND_RT,STWT,PMPT,GS_SPM2,P40H_COND_RT,RGX_RT,SRVNS,GS_BPOS,PMAX_RT,A28H_COND_RT,STUCK_RT,DCHR,DNISTAT,CRPM_RT,DBTM,GRID_CORR,STK4,SRV_HZ,ROP5,CRS1,ARC_GR_UNC_RT,GS_TV10,MAG_DEC,QROP,TOFB,SPM3,AZIM_CONT_RT,TVDE,OSTM,TSTK,SSAMP,GS_SPM3,ESD_MIN_RT,RSDX,SRV_GZ,GS_TV02,BITRUN,Stick_RT,PESD_RT,ROP30s,P28H_COND_RT,CRS_CNINC,G

Pupulating a dataframe with elements from the parsed XML file

In [114]:
# Importing packages
import numpy as np
import pandas as pd
from datetime import datetime

#Fetching the elements from the MnemonicList tag (log short-names) into a list
mnemonicList=root.find(".//{http://www.witsml.org/schemas/1series}mnemonicList").text
mnemonicList=mnemonicList.split(",")

#Fetching the elements from the UnitList tag (unirts for the data) into a list
unitList=root.find(".//{http://www.witsml.org/schemas/1series}unitList").text
unitList.split(",")

#Fetching the elements from the Data tag into a list
data=[]
for m in root.findall(".//{http://www.witsml.org/schemas/1series}data"):
    string=m.text
    data.append(string.split(","))
    
#Converting list of data elements to an numpy array (of strings)
log_data_array=np.asarray(data)

#Converting numpy array to a DataFrame with the element from MnemonicList as column names
df_log=pd.DataFrame(log_data_array, columns=mnemonicList)

#Replacing empty string elements to NaN
df_log=df_log.replace('', np.nan)

#All the numbers in the DataFrame are stored as strings. These elements should be numbers, with exception of the PASS_NAME (comments) and TIME columns
#Converting all elements to float64, except the PASS_NAME column
for key in df_log.keys():
    if key=='PASS_NAME':
        df_log[key]=df_log[key].astype('object')
    else:
      if key=='TIME':
        df_log[key]=df_log[key].astype('datetime64')
      else:
        try:
            df_log[key]=df_log[key].astype('float64')
        except:
            df_log.drop(key, axis=1) #Included try-except to avoid errors. Just dropping columns that won't convert

#Setting the time column as index
df_log=df_log.set_index('TIME')

C:\appl\Anaconda3\lib\site-packages\pandas\core\internals.py:3462: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


The relevant time series data should now be populated in the DataFrame df_log

In [115]:
df_log.head()

,SRVTVD,GS_SPM1,GS_TV11,SRV_HY,HKLD30s,UTSTAT,SWOB30s,GS_TV03,SPPA,ETSL,...,RPM,STKSLP,BP_CORR_P,SHKLV,GS_MDOA,SSFRQ,SPM4,RGX,GTF_RT,GS_G_IC4
TIME,,,,,,,,,,,,,,,,,,,,,
2009-01-22 00:01:26,NaN,78.0,2.06,NaN,164.613661,0.0,2.575951,53.849998,19797.178617,606.99,...,180.0,NaN,0.0,NaN,1.24,NaN,0.0,NaN,NaN,NaN
2009-01-22 00:01:29,NaN,NaN,NaN,NaN,164.589620,0.0,2.599991,NaN,19672.693774,606.99,...,179.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2009-01-22 00:01:30,NaN,78.0,2.06,NaN,NaN,NaN,NaN,53.860001,NaN,NaN,...,NaN,NaN,NaN,NaN,1.24,NaN,NaN,NaN,NaN,NaN
2009-01-22 00:01:32,NaN,NaN,NaN,NaN,164.549704,0.0,2.639908,NaN,19672.693774,606.99,...,179.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2009-01-22 00:01:35,NaN,78.0,2.06,NaN,164.506159,0.0,2.683452,53.849998,19672.693774,606.99,...,179.0,NaN,0.0,NaN,1.24,NaN,0.0,NaN,NaN,NaN


# Process data

### Downsample data

- By default the frequency of the data is 2 sec (regardless of sending value or not)
- Downsampling data to 30 sec interval, by taking the mean value in the interval

In [189]:
df=df_log.resample('0.5min').mean()

Use only columns with moe than 100 values

In [190]:
df=df[df.columns[df.describe().loc['count',:]>100]]

Check columns

In [191]:
df.columns

Index(['GS_SPM1', 'GS_TV11', 'HKLD30s', 'UTSTAT', 'SWOB30s', 'GS_TV03', 'SPPA',
       'ETSL', 'AJAM_MWD', 'ATMP_RT',
       ...
       'GS_SPPA', 'RPM', 'STKSLP', 'BP_CORR_P', 'SHKLV', 'GS_MDOA', 'SSFRQ',
       'SPM4', 'RGX', 'GS_G_IC4'],
      dtype='object', length=166)

Select following columns:
- GS_HKLD (hookload, weight of string)
- GS_TQA (torque)
- GS_MTOA (temprature of returning fluid)
- GS_SPPA (Standpipe pressure, pressure sensor fluid in)
- GS_TFLO (Flow of fluid in)
- GS_DVER (Vertical depth)

In [192]:
df=df[['GS_HKLD', 'GS_TQA','GS_MTOA', 'GS_SPPA', 'GS_TFLO', 'GS_DVER']]

Drop rows that have NaN-values in all columns. Also dropping first column

In [193]:
df=df.dropna(how='all')[1:]

Rename columns to more describing names

In [194]:
df.columns=['Hookload(Tonne)', 'Torque(kNm)', 'Temp_out(C)', 'Pressure(bar)', 'Flow(lpm)', 'Vertical Depth(m)']

In [202]:
df.head()

,Hookload(Tonne),Torque(kNm),Temp_out(C),Pressure(bar),Flow(lpm),Vertical Depth(m)
TIME,,,,,,
2009-01-22 00:01:30,171.586189,23.792857,61.570000,19720.428685,3401.354249,2697.882847
2009-01-22 00:02:00,171.257938,23.851666,61.690001,19750.333405,3400.984986,2698.239990
2009-01-22 00:02:30,171.310282,23.740000,61.780000,19737.599793,3401.589942,2698.239990
2009-01-22 00:03:00,171.103283,24.305000,61.719999,19717.499796,3401.426595,2698.239990
2009-01-22 00:03:30,170.835970,24.242857,61.817144,19747.714670,3401.014265,2698.239990


## The End

This is the end of this tutorial.